In [41]:
#load libraries for data wrangling
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import os
import dask.dataframe as dd
import csv
import importlib
import html
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import psutil
from collections import defaultdict
import pickle



#load function files
import Functions 
from Functions import * 

In [ ]:
importlib.reload(Functions)

### Data from scratch - Pandas

In [4]:
#How is my data delimited?
path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'



def print_lines(path, file, start_line=0, end_line=10):
    with open(f"{path}/{file}", 'r') as f:
        for i in range(end_line):
            line = f.readline()
            if i >= start_line:
                print(line)

print_lines(path, file)


#The data is comma delimited

twitter_id,id,screen_name,name,description,url,timestamp_utc,local_time,location,verified,protected,tweets,followers,friends,likes,lists,image,default_profile,default_profile_image,witheld_in_countries,witheld_scope

3342215494,3342215494,titisanogo8,Titi sanogo,Je crois en DIEU et à mon travail j'y arriverai.....,,1435017944,2015-06-23T00:05:44,"Ile-de-France, France",0,0,6,44,733,91,0,https://pbs.twimg.com/profile_images/1249394390029742081/xuVolLn6_normal.jpg,1,0,,

3115495713,3115495713,AndreDeybach,DEYBACH André,,,1427309108,2015-03-25T18:45:08,,0,0,0,1,40,0,0,https://pbs.twimg.com/profile_images/580803690757533697/pHNcCBLh_normal.jpg,1,0,,

244075010,244075010,matttownley1985,Matt Townley,"Hotelier, traveller, fan of all things hospitality, great food and fine wine! All views my own etc!!",,1296220595,2011-01-28T13:16:35,"Manchester, England",0,1,2535,772,1264,1251,7,https://pbs.twimg.com/profile_images/928075998930681856/ZFXboKc3_normal.jpg,0,0,,

2986463442,2986463442,alex_guev

In [ ]:
# #Loading in one data set
# path = '/home/livtollanes/SocialMarkers'
# file = 'markers_followers_bios_2023-05-19.csv'

# def load_file(path, file, req_cols):
#     return pd.read_csv(f"{path}/{file}", delimiter=',', quotechar='"', low_memory=False, usecols=req_cols)


# req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

# # dtypes = {
# #     'twitter_id': 'int64',
# #     'id': 'float64',
# #     'screen_name': 'object',
# #     'description': 'object',
# #     'location': 'object',
# #     'tweets': 'float64',
# #     'followers': 'float64',
# #     'friends': 'float64',
# #     'witheld_in_countries': 'float64'
# # }

# df = load_file(path, file, req_cols)

In [ ]:
# # How much memory available?
# def get_available_memory():
#     return psutil.virtual_memory().available

# available_memory = get_available_memory()
# print(f"Available memory: {available_memory / (1024 * 1024 * 1024)} GB")


# # check size of loaded df and print 
# def get_df_memory_usage(df):
#     return df.info(memory_usage='deep')

# get_df_memory_usage(df)

#### Data inspection

In [ ]:
# df.head()

# #insopect min and max value of column friends with f string
# # print(f"Min friends: {df.friends.min()}")
# # print(f"Max friends: {df.friends.max()}")


# #check for multiple lines per user
# duplicates = df.duplicated(subset='twitter_id', keep=False)
# print(f"Number of duplicate rows: {duplicates.sum()}")



## 1. Filtering out irrelevant users based on follower count

- In this part, we want to remove users that don't follow enough brands. 
- This is to match the procedure of He and Tsvetkova (2023)
- The purpose is to ensure we have enough information to generate SES estimates for users.

In [5]:
#How is my data delimited?
path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_2023-05-19.csv'



def print_lines(path, file, start_line=0, end_line=10):
    with open(f"{path}/{file}", 'r') as f:
        for i in range(end_line):
            line = f.readline()
            if i >= start_line:
                print(line)

print_lines(path, file)


id,cursor,follower_id

415859364,,1655336804831174657

415859364,,1659648141497454593

415859364,,1525534139478310915

415859364,,1659648883209674764

415859364,,1659648836594458626

415859364,,881616301

415859364,,1618696506424303637

415859364,,1659647872202055692

415859364,,1659647973637357568



In [ ]:
# what about the other dfs?
path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_2023-05-19.csv'



def print_lines(path, file, start_line=0, end_line=10):
    with open(f"{path}/{file}", 'r') as f:
        for i in range(end_line):
            line = f.readline()
            if i >= start_line:
                print(line)

print_lines(path, file)


#### Identify the number of brands each unique follower follows

In [ ]:
### Do the keys in the dict match the unique IDs in the file?

# # Load the data into a DataFrame
# df = pd.read_csv('/home/livtollanes/SocialMarkers/markers_followers_2023-05-19.csv',dtype={'cursor': object})

# df.head()

Create dictionary of [keys: follower_Id, Value: Brand_id]
Without loading the data into python

In [8]:
# Create a dictionary of keys:follower_id and value: brands
brands_per_follower = defaultdict(set)

# Open the CSV file
with open('/home/livtollanes/SocialMarkers/markers_followers_2023-05-19.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Add the brand id to the set of brands for this follower
        brands_per_follower[row['follower_id']].add(row['id'])


# Convert the sets to counts to see how many brands each follower follows
brands_per_follower_count = {follower_id: len(brands) for follower_id, brands in brands_per_follower.items()}

In [34]:
#How many unique values (brands) are there in the dictionary?

unique_values = set(value for values in brands_per_follower.values() for value in values)

# The number of unique values is the length of the set
num_unique_values = len(unique_values)

print(f"The number of unique values in the dictionary is {num_unique_values}.")

The number of unique values in the dictionary is 236.


In [11]:
# How many keys (follower_id) are there in the dictionary?
num_keys = len(brands_per_follower_count)

print(f"The number of keys in the dictionary is {num_keys}.")

The number of keys in the dictionary is 70636295.


In [32]:
#inspect first key value pairs 
# Get an iterator over the dictionary's items
items = iter(brands_per_follower_count.items())

# Get the first 5 items
for _ in range(10):
    print(next(items))

    

('1655336804831174657', 4)
('1659648141497454593', 3)
('1525534139478310915', 3)
('1659648883209674764', 1)
('1659648836594458626', 1)
('881616301', 1)
('1618696506424303637', 1)
('1659647872202055692', 1)
('1659647973637357568', 1)
('1659647339697524736', 1)


In [30]:
# How many users follow less than num_brands brands?
num_brands = 5

# How many users follow less than num_brands brands?
# Assuming brands_per_follower_count is your dictionary
count = sum(1 for value in brands_per_follower_count.values() if value < num_brands)

# Calculate the percentage
percentage = (count / len(brands_per_follower_count)) * 100

# Calculate the percentage of the remaining data
remaining_percentage = 100 - percentage

# Calculate the number of remaining users
remaining_users = len(brands_per_follower_count) - count

print(f"The number of keys that follow less than {num_brands} IDs is {count}, which is {percentage:.2f}% of the total. Removing these leaves {remaining_percentage:.2f}% of the data, or {remaining_users} users.")

The number of keys that follow less than 5 IDs is 66693204, which is 94.42% of the total. Removing these leaves 5.58% of the data, or 3943091 users.


In [39]:
# excluding users that follow less than 5 brands in our full dict

num_brands = 5

# Filter the dictionary
filtered_brands_per_follower = {follower_id: brands for follower_id, brands in brands_per_follower.items() if len(brands) >= num_brands}

#inspect first key value pairs in filtered dict
items = iter(filtered_brands_per_follower.items())

# Get the first 5 items
for _ in range(10):
    print(next(items))


('1543191014579617793', {'326359913', '88944305', '415859364', '38164846', '348379865'})
('1257705208592756737', {'60659498', '326359913', '60054156', '415859364', '348379865', '44084633'})
('1301293067945807877', {'554957168', '88944305', '21915474', '415859364', '44084633'})
('1644463375231885315', {'60659498', '60054156', '16096416', '88944305', '415859364', '348379865', '44084633'})
('1233001848006729729', {'112754792', '133663801', '40448264', '88944305', '83864876', '415859364', '33893706', '19976004', '47902100'})
('1659583369536053248', {'60054156', '16096416', '415859364', '348379865', '44084633'})
('835476283249287169', {'42497567', '537989713', '415859364', '16097522', '36383320'})
('2768568386', {'554957168', '415859364', '348379865', '44084633', '18481641'})
('1659538858864791552', {'554957168', '63142684', '61193820', '415859364', '348379865', '44084633'})
('923959702089797633', {'25487201', '861574608', '415859364', '38164846', '348379865', '44084633'})


In [42]:
# Create a list for the relevant user 
# Save the list to a file
with open('/home/livtollanes/NewData/followfive_filtered.pkl', 'wb') as f:
    pickle.dump(filtered_brands_per_follower, f)


#load the pickle list from file
# with open('/path/to/your/directory/followers_filtered.pkl', 'rb') as f:
#     followers_filtered = pickle.load(f)

### Other filterings to do
- sent at least 100 tweets
- have at least 25 followers
- sent at least five tweets in the first few months of the year the data was collected (maybe not relevant for this data)